Autores:

Yalim Villegas Polo,
Jhon Meneces Viecco.

In [4]:
from collections import deque
import time
import heapq


# **Importación de herramientas**

Se importaron las siguientes librerías:

- `time`: Se usa para cronometrar el tiempo de ejecución de los algoritmos.
- `heapq`: Implementa colas de prioridad, necesarias para el algoritmo A*.
- `deque` (del módulo `collections`): Proporciona una implementación eficiente de colas dobles, usada en BFS.

In [5]:
class Node():
    def __init__(self, state, parent, action, depth):
        self.state = state
        self.parent = parent
        self.action = action
        self.depth = depth

    def __str__(self):
        return f"Estado: {self.state}, Acción: {self.action}, Profundidad: {self.depth}"

    def print_state(self):
        for i in range(0, 9, 3):
            print(self.state[i], self.state[i+1], self.state[i+2])
        print()

# **Clase Node**

## Propósito
Clase base que representa un nodo en el árbol de búsqueda para el 8-puzzle, almacenando la información esencial del estado actual y su relación con otros nodos.

## Características principales

1. **Estructura fundamental**:
   - Almacena el estado actual del puzzle
   - Mantiene referencia al nodo padre
   - Registra la acción que llevó a este estado
   - Guarda la profundidad en el árbol de búsqueda

2. **Métodos de utilidad**:
   - Representación textual del nodo (__str__)
   - Visualización del estado del puzzle (print_state)

## Componentes clave

**Atributos**:
- `state`: Tupla que representa la configuración actual del 8-puzzle
- `parent`: Referencia al nodo padre en el árbol de búsqueda
- `action`: Movimiento ('up', 'down', 'left', 'right') que generó este estado
- `depth`: Profundidad del nodo en el árbol (costo desde el inicio)

**Método __str__**:
- Proporciona representación legible del nodo
- Muestra estado, acción y profundidad
- Útil para debugging y seguimiento del algoritmo

**Método print_state()**:
- Imprime el estado del puzzle en formato 3x3
- Facilita visualización del tablero en cada paso
- Incluye separación entre filas y espacio final


In [6]:

class AStarNode(Node):
    def __init__(self, state, parent, action, depth, cost):
        super().__init__(state, parent, action, depth)
        self.cost = cost
        
    def __lt__(self, other):
        return self.cost < other.cost

# **Clase AStarNode**

## Propósito
Extiende la clase `Node` para implementar nodos especializados en el algoritmo A*, incorporando el concepto de costo total (f = g + h) que combina el costo real desde el inicio con la heurística estimada al objetivo.

## Características principales

1. **Extensión de Node**:
   - Hereda todos los atributos base (state, parent, action, depth)
   - Añade el nuevo atributo `cost` para almacenar el costo total f(n)

2. **Método __lt__**:
   - Sobreescribe el operador de comparación `<`
   - Permite la ordenación automática de nodos por costo
   - Esencial para el funcionamiento de la cola de prioridad en A*

## Componentes clave

**Atributo cost**:
- Almacena el valor f(n) = g(n) + h(n)
- g(n): Costo real desde el inicio (normalmente la profundidad)
- h(n): Valor heurístico (ej: distancia Manhattan)

**Comparación entre nodos**:
- Define cómo se ordenan los nodos en la frontera de búsqueda
- Los nodos con menor costo tienen mayor prioridad
- Permite a heapq ordenar eficientemente los nodos

## Importancia en A*

1. **Para la cola de prioridad**:
   - La función __lt__ permite que heapq compare nodos
   - Mantiene siempre el nodo con menor costo en la cima

2. **Para la optimalidad**:
   - Garantiza que siempre se expanda primero el nodo más prometedor
   - Es fundamental para que A* encuentre la solución óptima

3. **Para la eficiencia**:
   - Reduce el número de nodos expandidos
   - Dirige la búsqueda hacia los caminos más eficientes

## Relación con el algoritmo A*

- Cada nodo generado calcula su costo total
- La cola de prioridad usa __lt__ para ordenar
- El nodo con menor costo se expande primero
- Esto combina lo mejor de UCS y búsqueda greedy


In [7]:

class PuzzleSolver():
    def __init__(self, initial_state, goal_state):
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.solution = None
        self.nodes_expanded = 0
        self.max_depth = 0
        
    def is_goal(self, state):
        return state == self.goal_state
    
    def get_blank_position(self, state):
        return state.index(0)
    

# **Clase PuzzleSolver**

Esta clase resuelve el 8-puzzle (rompecabezas deslizante 3x3) implementando algoritmos de búsqueda clásicos.

**Atributos**:
- `initial_state` y `goal_state`: Definen el problema (ej: `(1,2,3,4,5,6,7,8,0)`)
- `solution`: Almacena la secuencia de movimientos solución
- `nodes_expanded`: Cuenta eficiencia (nodos explorados)
- `max_depth`: Controla profundidad de búsqueda

**Mecanismo básico**:
1. Inicializa con los estados inicial y objetivo
2. Los algoritmos (BFS/DFS/A*) usan:
   - `is_goal()` para verificar solución
   - `get_blank_position()` para generar movimientos
3. Actualiza métricas durante la búsqueda

In [8]:

    def get_possible_moves(self, state):
        blank_pos = self.get_blank_position(state)
        moves = []
        
        if blank_pos > 2:
            new_state = list(state)
            new_state[blank_pos], new_state[blank_pos-3] = new_state[blank_pos-3], new_state[blank_pos]
            moves.append(('up', tuple(new_state)))
 
        if blank_pos < 6:
            new_state = list(state)
            new_state[blank_pos], new_state[blank_pos+3] = new_state[blank_pos+3], new_state[blank_pos]
            moves.append(('down', tuple(new_state)))
 
        if blank_pos % 3 > 0:
            new_state = list(state)
            new_state[blank_pos], new_state[blank_pos-1] = new_state[blank_pos-1], new_state[blank_pos]
            moves.append(('left', tuple(new_state)))
 
        if blank_pos % 3 < 2:
            new_state = list(state)
            new_state[blank_pos], new_state[blank_pos+1] = new_state[blank_pos+1], new_state[blank_pos]
            moves.append(('right', tuple(new_state)))
            
        return moves
    
    

# **Método get_possible_moves**  
Genera todos los movimientos válidos del espacio vacío (0) en el puzzle 3x3.

##  Cómo funciona
1. **Localiza el 0** → Usa `get_blank_position(state)`  
2. **Verifica límites** para cada movimiento:

| Movimiento | Condición           | Límites del tablero              | Ejemplo (posición 4) |
|------------|---------------------|-----------------------------------|-----------------------|
| **↑ Arriba**   | `blank_pos > 2`     | No en fila superior (pos 0-2)     | 4 > 2 → ✅ Válido      |
| **↓ Abajo**    | `blank_pos < 6`     | No en fila inferior (pos 6-8)     | 4 < 6 → ✅ Válido      |
| **← Izquierda**| `blank_pos % 3 > 0` | No en columna izquierda (0,3,6)   | 4%3=1 > 0 → ✅ Válido  |
| **→ Derecha**  | `blank_pos % 3 < 2` | No en columna derecha (2,5,8)     | 4%3=1 < 2 → ✅ Válido  |

##  Casos especiales
- **Esquina (pos 0)**: Solo ↓ y →  
- **Centro (pos 4)**: Todos los movimientos  
- **Borde derecho (pos 5)**: ↑, ↓, ←  

##  Qué devuelve
Lista de tuplas con:  
```python 
[('dirección', nuevo_estado), ...]  

In [9]:
def bfs_solve(self):
        start_time = time.time()
        frontier = deque()
        explored = set()
        
        initial_node = Node(self.initial_state, None, None, 0)
        frontier.append(initial_node)
        
        while frontier:
            current_node = frontier.popleft()
            self.nodes_expanded += 1
            self.max_depth = max(self.max_depth, current_node.depth)
            
            if self.is_goal(current_node.state):
                self.solution = self.get_solution_path(current_node)
                end_time = time.time()
                return {
                    'time': end_time - start_time,
                    'nodes_expanded': self.nodes_expanded,
                    'solution_length': len(self.solution),
                    'max_depth': self.max_depth
                }
            
            explored.add(current_node.state)
            
            for action, state in self.get_possible_moves(current_node.state):
                if state not in explored and not any(node.state == state for node in frontier):
                    child_node = Node(state, current_node, action, current_node.depth + 1)
                    frontier.append(child_node)
        
        return None
    
   

# **Método bfs_solve()**

## Funcionamiento
Este método implementa el algoritmo Breadth-First Search (BFS) para resolver el 8-puzzle. Explora sistemáticamente todos los posibles estados del puzzle nivel por nivel, garantizando encontrar la solución más corta si existe.

## Lógica del algoritmo
1. **Inicialización**: 
   - Crea una cola (frontier) para almacenar nodos pendientes de exploración
   - Utiliza un conjunto (explored) para registrar estados ya visitados
   - Comienza con el estado inicial en la cola

2. **Proceso principal**:
   - Extrae el primer nodo de la cola (FIFO)
   - Verifica si es el estado objetivo
   - Si es solución, reconstruye el camino y retorna métricas
   - Si no, genera todos los movimientos válidos y añade nuevos estados a la cola

## Puntos clave y comandos importantes

1. **deque()**: 
   - Se usa para la cola frontier porque ofrece operaciones O(1) para append/popleft
   - Esencial para la naturaleza FIFO de BFS

2. **set() para explored**:
   - Permite verificación rápida (O(1)) de estados ya visitados
   - Evita ciclos y procesamiento redundante

3. **popleft()**:
   - Extrae el nodo más antiguo de la cola
   - Esto garantiza la exploración por niveles (característica principal de BFS)

4. **any(node.state == state for node in frontier)**:
   - Verifica eficientemente si un estado ya está en la cola
   - Evita duplicados antes de añadir nuevos nodos

5. **Métricas registradas**:
   - Tiempo de ejecución: mide eficiencia del algoritmo
   - Nodos expandidos: indica complejidad de la solución
   - Longitud de solución: muestra optimalidad (siempre la más corta en BFS)
   - Profundidad máxima: ayuda a entender el uso de memoria

## Características importantes
- **Completitud**: Siempre encuentra solución si existe
- **Optimalidad**: Devuelve la solución con menor número de movimientos
- **Complejidad de memoria**: O(b^d), donde b es el factor de ramificación y d la profundidad de la solución
- **Estrategia**: Exploración uniforme en amplitud antes de profundidad


In [10]:
 def dfs_solve(self, depth_limit=30):
        start_time = time.time()
        frontier = []
        explored = set()
        
        initial_node = Node(self.initial_state, None, None, 0)
        frontier.append(initial_node)
        
        while frontier:
            current_node = frontier.pop()
            self.nodes_expanded += 1
            self.max_depth = max(self.max_depth, current_node.depth)
            
            if self.is_goal(current_node.state):
                self.solution = self.get_solution_path(current_node)
                end_time = time.time()
                return {
                    'time': end_time - start_time,
                    'nodes_expanded': self.nodes_expanded,
                    'solution_length': len(self.solution),
                    'max_depth': self.max_depth
                }
            
            explored.add(current_node.state)
            
            if current_node.depth < depth_limit:
                for action, state in self.get_possible_moves(current_node.state):
                    if state not in explored and not any(node.state == state for node in frontier):
                        child_node = Node(state, current_node, action, current_node.depth + 1)
                        frontier.append(child_node)
        
        return None
    

# **Método dfs_solve()**

## Funcionamiento
Este método implementa el algoritmo Depth-First Search (DFS) para resolver el 8-puzzle con un límite de profundidad. Explora cada rama del árbol de búsqueda completamente antes de retroceder, utilizando una pila para gestionar los nodos por visitar.

## Lógica del algoritmo
1. **Inicialización**:
   - Utiliza una lista (frontier) como pila (LIFO)
   - Mantiene un conjunto (explored) de estados visitados
   - Comienza con el estado inicial en la pila

2. **Proceso principal**:
   - Extrae el último nodo añadido a la pila (LIFO)
   - Verifica si es el estado objetivo
   - Si es solución, reconstruye el camino y retorna métricas
   - Si no, genera movimientos válidos dentro del límite de profundidad

## Puntos clave y comandos importantes

1. **Lista como pila**:
   - Usa append() y pop() para operaciones LIFO
   - Esto implementa la estrategia de profundidad primero

2. **depth_limit**:
   - Parámetro crítico que evita búsqueda infinita
   - Valor por defecto de 30 niveles como seguridad

3. **pop()**:
   - Extrae el nodo más reciente de la pila
   - Esto garantiza la exploración en profundidad

4. **Condición current_node.depth < depth_limit**:
   - Restringe la exploración a profundidades razonables
   - Previene el desbordamiento de memoria

5. **Métricas registradas**:
   - Tiempo de ejecución: útil para comparar rendimiento
   - Nodos expandidos: muestra esfuerzo computacional
   - Longitud de solución: puede no ser óptima (DFS)
   - Profundidad máxima: importante con límite aplicado

## Características importantes
- **Estrategia**: Profundidad primero (explora ramas completas)
- **Memoria**: O(b*d) donde b es factor de ramificación y d profundidad
- **Completitud**: Solo con límite de profundidad
- **Optimalidad**: No garantiza solución más corta
- **Ventaja**: Menor uso de memoria que BFS para algunos casos


In [11]:

    def manhattan_distance(self, state):
        distance = 0
        for i in range(9):
            if state[i] != 0:  # Skip the blank tile
                current_row, current_col = i // 3, i % 3
                goal_index = self.goal_state.index(state[i])
                goal_row, goal_col = goal_index // 3, goal_index % 3
                distance += abs(current_row - goal_row) + abs(current_col - goal_col)
        return distance
    

# **Método manhattan_distance()**

## Funcionamiento
Este método calcula la heurística de distancia Manhattan para el 8-puzzle, que estima el costo mínimo para llegar al estado objetivo sumando las distancias horizontales y verticales de cada ficha a su posición correcta.

## Lógica del cálculo
1. **Inicialización**:
   - Comienza con distancia total en 0
   - Ignora el espacio vacío (0) en el cálculo

2. **Proceso principal**:
   - Para cada ficha en el estado actual:
     1. Determina su posición actual (fila, columna)
     2. Encuentra su posición objetivo
     3. Suma las diferencias absolutas de filas y columnas

## Puntos clave y operaciones importantes

1. **i // 3 y i % 3**:
   - Convierten el índice lineal (0-8) a coordenadas (fila, columna)
   - Ejemplo: índice 5 → fila 1 (5//3), columna 2 (5%3)

2. **goal_state.index(state[i])**:
   - Encuentra la posición correcta de la ficha en el estado objetivo
   - Base para calcular la distancia a la posición ideal

3. **abs(current_row - goal_row)**:
   - Calcula distancia vertical entre posición actual y objetivo
   - Parte esencial de la métrica Manhattan

4. **abs(current_col - goal_col)**:
   - Calcula distancia horizontal entre posición actual y objetivo
   - Complementa la distancia vertical

5. **Exclusión del espacio vacío (0)**:
   - No contribuye a la distancia total
   - Mejora la eficiencia del cálculo


In [12]:

    def astar_solve(self):
        start_time = time.time()
        frontier = []
        explored = set()
        
        initial_cost = self.manhattan_distance(self.initial_state)
        initial_node = AStarNode(self.initial_state, None, None, 0, initial_cost)
        heapq.heappush(frontier, (initial_cost, initial_node))
        
        while frontier:
            _, current_node = heapq.heappop(frontier)
            self.nodes_expanded += 1
            self.max_depth = max(self.max_depth, current_node.depth)
            
            if self.is_goal(current_node.state):
                self.solution = self.get_solution_path(current_node)
                end_time = time.time()
                return {
                    'time': end_time - start_time,
                    'nodes_expanded': self.nodes_expanded,
                    'solution_length': len(self.solution),
                    'max_depth': self.max_depth
                }
            
            explored.add(current_node.state)
            
            for action, state in self.get_possible_moves(current_node.state):
                if state not in explored:
                    new_depth = current_node.depth + 1
                    cost = new_depth + self.manhattan_distance(state)
                    child_node = AStarNode(state, current_node, action, new_depth, cost)
                    heapq.heappush(frontier, (cost, child_node))
        
        return None
    

# **Método astar_solve()**

## Funcionamiento
Implementa el algoritmo A* para resolver el 8-puzzle usando la heurística de distancia Manhattan. Combina el costo real desde el inicio (g(n)) con la estimación al objetivo (h(n)) para priorizar nodos prometedores.

## Lógica del algoritmo
1. **Inicialización**:
   - Crea una cola de prioridad (frontier) usando `heapq`
   - Calcula costo inicial (f = g + h) usando `manhattan_distance`
   - Añade nodo inicial a la frontera priorizada

2. **Proceso principal**:
   - Extrae el nodo con menor costo estimado (f) de la frontera
   - Verifica si es estado objetivo
   - Expande movimientos válidos, calculando nuevo costo f(n) = g(n) + h(n)
   - Añade nuevos nodos a la frontera si no fueron explorados

## Componentes clave

1. **heapq**:
   - Estructura eficiente para manejar la cola de prioridad
   - Garantiza extracción del nodo con menor costo en O(log n)

2. **Costo f(n) = g(n) + h(n)**:
   - `g(n)`: Profundidad del nodo (costo real desde inicio)
   - `h(n)`: Distancia Manhattan (estimación al objetivo)

3. **AStarNode**:
   - Almacena estado, padre, acción, profundidad y costo total
   - Permite reconstrucción del camino solución

4. **Exploración eficiente**:
   - `explored` evita reprocesamiento de estados
   - Prioriza nodos con menor f(n) para búsqueda óptima

## Datos retornados
```python
{
    'time': float,          # Tiempo total de ejecución
    'nodes_expanded': int,  # Total nodos explorados
    'solution_length': int, # Pasos de la solución
    'max_depth': int        # Profundidad máxima alcanzada
}

In [13]:

    def get_solution_path(self, node):
        path = []
        while node.parent is not None:
            path.append((node.action, node.state))
            node = node.parent
        path.reverse()
        return path
    
    def print_solution(self):
        if not self.solution:
            print("No se encontró solución")
            return
            
        print("Estado inicial:")
        Node(self.initial_state, None, None, 0).print_state()
        
        for step, (action, state) in enumerate(self.solution, 1):
            print(f"Paso {step}: Mover {action}")
            Node(state, None, None, 0).print_state()
        
        print(f"Solución encontrada en {len(self.solution)} movimientos")
        print(f"Nodos expandidos: {self.nodes_expanded}")
        print(f"Profundidad máxima alcanzada: {self.max_depth}")

# **Métodos de Visualización**

## get_solution_path(node)

### Funcionalidad:
Reconstruye el camino solución desde el nodo objetivo hasta el nodo inicial.

### Proceso:
1. **Inicialización**: Crea una lista vacía para almacenar el camino
2. **Recorrido hacia atrás**:
   - Comienza desde el nodo solución
   - Sigue los enlaces `parent` hasta llegar al nodo raíz
   - Almacena cada (acción, estado) en el camino
3. **Ajuste del orden**:
   - Invierte la lista para presentar el camino en orden inicio→solución
   - Retorna la secuencia ordenada

## print_solution()

### Funcionalidad:
Muestra la solución encontrada de forma legible con métricas de rendimiento.

### Flujo de ejecución:
1. **Verificación de solución**:
   - Si no existe solución, muestra mensaje y termina
2. **Mostrar estado inicial**:
   - Imprime la configuración inicial del puzzle
3. **Pasos de solución**:
   - Enumera cada movimiento con su estado resultante
4. **Métricas finales**:
   - Total de movimientos de la solución
   - Nodos expandidos durante la búsqueda
   - Profundidad máxima del árbol de búsqueda

### Formatos de visualización:
- **Estado del puzzle**: Matriz 3x3 legible
- **Pasos**: Numerados secuencialmente
- **Acciones**: Texto descriptivo (up/down/left/right)


In [14]:

def main():
    #initial_state = (1, 2, 3, 4, 5, 6, 7, 0, 8) #facil
    #initial_state = (8, 6, 7, 2, 5, 4, 3, 0, 1) #dificil
    initial_state = (0, 1, 2, 3, 5, 6, 4, 7, 8)
    goal_state = (1, 2, 3, 4, 5, 6, 7, 8, 0)
    
    print("Resolviendo con BFS...")
    bfs_solver = PuzzleSolver(initial_state, goal_state)
    bfs_result = bfs_solver.bfs_solve()
    bfs_solver.print_solution()
    
    print("\nResolviendo con DFS...")
    dfs_solver = PuzzleSolver(initial_state, goal_state)
    dfs_result = dfs_solver.dfs_solve()
    dfs_solver.print_solution()
    
    print("\nResolviendo con A* (Manhattan)...")
    astar_solver = PuzzleSolver(initial_state, goal_state)
    astar_result = astar_solver.astar_solve()
    astar_solver.print_solution()
    
    print("\nComparación:")
    print(f"{'Métrica':<20} {'BFS':<15} {'DFS':<15} {'A*':<15}")
    print(f"{'Tiempo (s)':<20} {bfs_result['time']:<15.4f} {dfs_result['time']:<15.4f} {astar_result['time']:<15.4f}")
    print(f"{'Nodos expandidos':<20} {bfs_result['nodes_expanded']:<15} {dfs_result['nodes_expanded']:<15} {astar_result['nodes_expanded']:<15}")
    print(f"{'Longitud solución':<20} {bfs_result['solution_length']:<15} {dfs_result['solution_length']:<15} {astar_result['solution_length']:<15}")
    print(f"{'Profundidad máxima':<20} {bfs_result['max_depth']:<15} {dfs_result['max_depth']:<15} {astar_result['max_depth']:<15}")


# **Función main()**

## Propósito
La función main() ejecuta una comparativa sistemática entre tres algoritmos de resolución del 8-puzzle (BFS, DFS y A*), permitiendo evaluar su rendimiento bajo las mismas condiciones iniciales.

## Proceso de ejecución

1. **Configuración inicial**:
   - Establece un estado inicial de prueba (configuración intermedia)
   - Define el estado objetivo estándar del puzzle

2. **Evaluación de algoritmos**:
   - Para cada algoritmo (BFS, DFS, A*):
     * Crea una nueva instancia del solver
     * Ejecuta el algoritmo correspondiente
     * Captura las métricas de rendimiento
     * Muestra el detalle de la solución encontrada

3. **Análisis comparativo**:
   - Organiza los resultados en una tabla estructurada
   - Presenta cuatro métricas clave para cada método

## Características clave

**Mecanismo de comparación**:
- Usa el mismo estado inicial para todos los algoritmos
- Aplica idénticas condiciones de evaluación
- Genera salida estandarizada para comparación justa

**Métricas reportadas**:
1. Tiempo de ejecución (precisión de 4 decimales)
2. Número de nodos expandidos
3. Longitud de la secuencia solución
4. Profundidad máxima del árbol de búsqueda

## Estructura de salida

1. **Detalle por algoritmo**:
   - Muestra el estado inicial
   - Lista todos los movientos de la solución
   - Presenta las métricas específicas

2. **Tabla comparativa final**:

In [ ]:

Comparación:
Métrica              BFS             DFS             A*
Tiempo (s)           0.0637          0.0292          0.0005
Nodos expandidos     1632            11666           36
Longitud solución    12              28              12
Profundidad máxima   12              30              12

# **Resultados obtenidos**

**Observaciones principales:**
1. **Eficiencia notable de A***:
   - Sorprende gratamente cómo A* resolvió el problema con solo 36 nodos expandidos
   - El tiempo de ejecución (0.0005s) demuestra su superioridad práctica

2. **Comparación interesante BFS-DFS**:
   - BFS mantuvo un buen equilibrio entre tiempo y calidad de solución
   - DFS, aunque rápido (0.0292s), produjo una solución menos óptima (28 movimientos)

3. **Calidad de soluciones**:
   - Tanto BFS como A* encontraron la solución óptima (12 movimientos)
   - DFS mostró la limitación de no garantizar optimalidad

**Aprendizajes clave:**
- La heurística bien diseñada (distancia Manhattan) marca una diferencia abismal
- Cada algoritmo tiene sus fortalezas según el contexto de uso
- La teoría sobre complejidad se ve reflejada claramente en la práctica

**Valoración personal:**
"Implementar y comparar estos algoritmos ha sido una experiencia educativa valiosa. Ver numéricamente las diferencias de rendimiento ayuda a comprender mejor cuándo usar cada enfoque. Me quedo con la importancia de elegir el algoritmo adecuado para cada problema."
